# 모델 평가

### 1. 모델 로드 및 평가 데이터 전처리

In [1]:
import transformers
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW, BitsAndBytesConfig
from tqdm.notebook import tqdm
from nltk.translate.bleu_score import sentence_bleu

## Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

## 모델과 토크나이저의 경로
model_path = './checkpoints/checkpoint-16000'

## 모델과 토크나이저 로드
model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)

model.to(device)

Using device: cpu


/Users/jeeho/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/jeeho/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [6]:
## 데이터 불러오기
df_val = pd.read_csv("./중노년층_한국어_방언_데이터/data_2인발화.csv")

## 혹시모를 결측치 제거하기
df_val = df_val.dropna(subset=['standard', 'dialect'])

## 문장 길이 4-64로 조정하기
df_val['form_len'] = df_val.standard.apply(lambda x : len(str(x)))
print(df_val)
df_val = df_val[(df_val.form_len > 3) & (df_val.form_len <= 64)]
print(df_val)

       Unnamed: 0                                            dialect  \
0               0  제사를 모시고 있는데 할머니 할아버지 제사까지 다 모시고 자식 없는 큰 아비 제사까...   
1               1  그랑께 메느리가 교회 권사가 돼갖고 안 하려고 하는데 어찌게 집 안에서 이래라 저래...   
2               2    그랑께 내가 해놓고 제가 와서 기도나 하고 그러라고 가정 편하게 하려고 그라고 삽니다   
3              17  봄 되면 친구들이랑 꽃구경 자주 가지요 삼월달에도 가고 사월달에 사월 오일이면 벚꽃...   
4              20  그전에는 처음에는 아기들을 시누이들 살고 같이 살 때는 한 번씩 했는데 지금은 화투...   
...           ...                                                ...   
36692       67492               내장산 선운사 관악산 월출산 어 주로 가을에 단풍놀이를 많이 가제   
36693       67493                          나는 여그 읍에서 이 도시에서 사는 거이 좋소   
36694       67494  시골에 삼스로 부잣집 시집 와 갖고 어찌께 일이 많은가라 논밭이 많은가 어른은 일을...   
36695       67495                        그랑께 지금은 참 대통령 시상 살고 있소 편안하니   
36696       67496             그랑께 나는 이 도시가 좋소야 너무 고생해서 일을 못 해갖고 고생해서   

                                                standard  form_len  
0      제사를 모시고 있는데 할머니 할아버지 제사까지 다 모시고 자식 없는 큰 아비 제사까...       141

### 2. Corpus_BLEU(100문장) 평가

In [16]:
from nltk.translate.bleu_score import corpus_bleu
from transformers import PreTrainedTokenizerFast
from transformers import pipeline

# 번역을 위한 파이프라인 생성하기
translation_pipeline = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=-1,
    max_length = 64
)

dialect_token = "[방언]"
standard_token = "[표준]"

# 모든 번역을 저장할 리스트
dialect_targets = []
standard_targets = []
dialect_predictions = []
standard_predictions = []
i = 1

for index, row in df_val.head(100).iterrows():
    dialect = row['dialect']
    standard = row['standard']

    ## 방언 -> 표준어 번역 수행
    predicted_sentence_dialect = translation_pipeline(dialect_token + " " + dialect)[0]['translation_text']

    ## 표준어 -> 방언 번역 수행
    predicted_sentence_standard = translation_pipeline(standard_token + " " + standard)[0]['translation_text']

    # 토크나이저를 사용하여 문장을 토큰화
    dialect_tokens = tokenizer.tokenize(dialect)
    standard_tokens = tokenizer.tokenize(standard)
    predicted_tokens_dialect = tokenizer.tokenize(predicted_sentence_dialect)
    predicted_tokens_standard = tokenizer.tokenize(predicted_sentence_standard)

    # 토큰화된 문장을 리스트에 추가
    dialect_targets.append([dialect_tokens])  # 참조는 리스트의 리스트가 되어야 함
    standard_targets.append([standard_tokens])  # 참조는 리스트의 리스트가 되어야 함
    dialect_predictions.append(predicted_tokens_dialect)
    standard_predictions.append(predicted_tokens_standard)
    print(f'sentence {i} translated')
    i += 1

# corpus_bleu 함수를 사용한 BLEU 점수 계산
from_dialect_to_standard_bleu_score = corpus_bleu(standard_targets, dialect_predictions)
from_standard_to_dialect_bleu_score = corpus_bleu(dialect_targets, standard_predictions)
print(f"방언 -> 표준어 번역 BLEU Score : {from_dialect_to_standard_bleu_score}")
print(f"표준어 -> 방언 번역 BLEU Score : {from_standard_to_dialect_bleu_score}")
     

sentence 1 translated
sentence 2 translated
sentence 3 translated
sentence 4 translated
sentence 5 translated
sentence 6 translated
sentence 7 translated
sentence 8 translated
sentence 9 translated
sentence 10 translated
sentence 11 translated
sentence 12 translated
sentence 13 translated
sentence 14 translated
sentence 15 translated
sentence 16 translated
sentence 17 translated
sentence 18 translated
sentence 19 translated
sentence 20 translated
sentence 21 translated
sentence 22 translated
sentence 23 translated
sentence 24 translated
sentence 25 translated
sentence 26 translated
sentence 27 translated
sentence 28 translated
sentence 29 translated
sentence 30 translated
sentence 31 translated
sentence 32 translated
sentence 33 translated
sentence 34 translated
sentence 35 translated
sentence 36 translated
sentence 37 translated
sentence 38 translated
sentence 39 translated
sentence 40 translated
sentence 41 translated
sentence 42 translated
sentence 43 translated
sentence 44 translat

### 3. 단일 문장 번역 결과 확인

In [27]:
from transformers import pipeline

# 번역을 위한 파이프라인 생성
translation_pipeline = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=-1
)

dialect_token = "[방언]"
standard_token = "[표준]"

# 특정 문장 번역 예시
input_sentence = "빨래를 얼마나 한다고 저런 것을 사버렸어"  # 예시 입력 문장
translated_sentence = translation_pipeline(standard_token + " " + input_sentence)[0]['translation_text']
print('input_sentence :', input_sentence)
print("표준어 -> 방언:", translated_sentence)
print()

input_sentence = "빨래를 을매나 헌다고 저런 것을 사부렀어"  # 예시 입력 문장
translated_sentence = translation_pipeline(dialect_token + " " + input_sentence)[0]['translation_text']
print('input_sentence :', input_sentence)
print("방언 -> 표준어:", translated_sentence)

input_sentence : 빨래를 얼마나 한다고 저런 것을 사버렸어
표준어 -> 방언: 빨래를 을매나 한다고 저런 것을 사부렀으

input_sentence : 빨래를 을매나 헌다고 저런 것을 사부렀어
방언 -> 표준어: 빨래를 얼마나 한다고 저런 것을 사버렸어
